In [53]:
from datetime import datetime

In [ ]:
class FoodItem:
    def __init__(self, name, id, expiry, nutrients, quantity): 
        self.name = name 
        self.expiry = expiry
        self.nutrients = nutrients
        self.quantity = quantity
        self.id = id
    
    def __str__(self):
        return (f"ID: {self.id}\n"
            f"Name: {self.name}\n"
            f"Expiry: {self.expiry.date()}\n"
            f"Quantity: {self.quantity}\n"
            f"Nutrients: {self.nutrients}")



In [55]:
class Inventory:
    def __init__(self): 
        self.items = {}
    def add_item(self, id, name, expiry, nutrients, quantity):
        expiry = datetime.strptime(expiry, "%Y-%m-%d")
        newItem = FoodItem(name, id, expiry, nutrients, quantity)
        self.items[id] = newItem
        
    def search_item(self, name):
        name = name.upper()
        if self.items != None:
            for indiv_item in self.items.keys():
                if indiv_item.upper() == name: 
                    return self.items[name]
    def display_inventory(self):
        if self.items == None: 
            print("Inventory is empty.")
        else: 
            for item_id, food_item in self.items.items():
                print(f"{item_id} : {food_item}")

                
    def get_details(self):
        name =  input("Enter the name of the food item:")
        id = input("Enter ID")
        quantity = int(input("Enter quantity:"))
        expiry = input("Enter expiry date (YYYY-MM-DD): ")
        print("Enter nutritional info:")
        calories = int(input("  Calories: "))
        protein = int(input("  Protein (grams): ")) 
        vitamins = input("Vitamins it is rich in(seperate it with comma)")
        vitamins_list = [v.strip().upper() for v in vitamins.split(",") if v.strip()] 
        nutrients = {"calories": calories, "protein": protein, "vits" : vitamins_list}
        return id, name, expiry, nutrients, quantity
        

In [56]:

inventory = Inventory()
while True: 
    print("Choose an action:")
    inp = int(input("1. Add an item \n 2.Search an item \n 3.Delete an item \n 4.View inventory \n 5. Exit"))
    if inp == 1:
        id, name, expiry, nutrients, quantity = inventory.get_details()
        inventory.add_item(id, name, expiry, nutrients, quantity)
        break
inventory.display_inventory()

        

Choose an action:
Enter nutritional info:


AttributeError: 'FoodItem' object has no attribute 'item_id'